Este proyecto tiene como objetivo desarrollar un modelo de procesamiento
de lenguaje natural (NLP) capaz de generar resúmenes clínicos automáticos
a partir de un dataset de alrededor de 1700 conversaciones entre doctores y
sus pacientes, junto con los respectivos resúmenes y anotaciones.

    Los objetivos de esta entrega 3 son:

    1. Correciones de la entrega 2:

Por ejemplo, ¿por qué lematizais? ¿Habéis analizado qué pasa con los word embeddings lematizado vs no-lematizado? Para embeddings la recomendación es no lematizar y con tf-idf habría que analizarlo con la tarea que queráis resolver. Además, ¿qué son lo que vosotros denomiáis tokens? Porque de 4367 palabras únicas no sé cómo salen 173,867 tokens.

Por otro lado, el análisis de longitud está muy bien pero lo hacéis a nivel de palabra, no de token. De cara a siguientes entregas hacerlo también a nivel de token para ver si un BIOBert por ejemplo tiene contexto suficiente.



    2. Definición de la tarea:
¿Generación de resúmenes? ¿Clasificación por categorías? ¿Generar texto clínico? ¿Detectar entidades? 
Elegir 2.

    3. Tareas Específicas de la entrega 3

Para ello, se deberán usar técnicas tanto de Shallow ML (o ML tradicional), como algunos de los modelos de CNNs o Redes Recurrentes que hemos visto en clase.

Comparar experimentos usando distintas métricas y optimizar los hiperparámetros.

Usar atención, combinar features (no creo que aplique a nuestro problema)

    Mínimos exigibles:
• Dos técnicas de Shallow Learning utilizando técnicas de representación dispersa/sparse.
• Dos técnicas de Deep Learning comparando diferentes tipos de embeddings y fine-tuneandolos dependiendo del caso. Ejemplos:
• Word2Vec congelado vs Word2Vec fine-tuneado vs Word2Vec “from scratch”
• Embedding fine-tuneado durante el entrenamiento vs Embedding inicializado
• Comparar al menos dos formas de embeddings de cada tipo:
• Tradicionales: e.g., Bag-of-Words, TF-IDF, etc.
• Semánticos No-Contextuales: e.g., Glove, FastText, Word2Vec, etc.
• Contextuales: e.g., ELMo, BERT, Modelos pre-entrenados de Hugging-Face, etc.